# Init Library

In [1]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

from scipy import signal, fftpack
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation, Dropout
from keras import utils
utils.to_categorical

from tensorflow.keras.utils import to_categorical

# Load directory

In [2]:
load = "E:/Sekolah/Telkom/TA/LTSM/"
save = "E:/Sekolah/Telkom/TA/LTSM/Img/"
proc_30 = 'E:/Sekolah/Telkom/TA/LTSM/Process/30%/*.csv'
proc_50 = 'E:/Sekolah/Telkom/TA/LTSM/Process/50%/*.csv'
proc_70 = 'E:/Sekolah/Telkom/TA/LTSM/Process/70%/*.csv'
proc_90 = 'E:/Sekolah/Telkom/TA/LTSM/Process/90%/*.csv'
combine = 'E:/Sekolah/Telkom/TA/LTSM/Process/Comb/*.csv'

# Parameter

In [3]:
lowcut = 2.0
highcut = 10.0
fs = 100.0
order = 5

# Apply Parameter

In [4]:
files_csv =[]
for root, dirs, files in os.walk(load):
    for file_name in files:
        if file_name.endswith(".csv"):
            file_load = os.path.join(root, file_name)
            files_csv.append(file_load)

In [5]:
i = 0

for file_csv in files_csv:
   
    df = pd.read_csv(file_csv)
    var = df['X'].values
    data = df['Y'].values
     
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    
    #Filter bandpass
    filtered_signal = signal.filtfilt(b, a, data)
    
    # Rectify signal
    rectified_signal = np.abs(filtered_signal)
    
    # One-sided FFT
    signal_fft = np.fft.fft(filtered_signal)
    signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
    n = len(filtered_signal)
    freq = signal_freq[0:int(n/2)]
    fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2
    
    # One-sided PSD
    psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
    freq_psd = signal_freq[0:int(n/2)]
    psd_one_side = psd[0:int(n/2)]
    
    processed_csv_file = file_csv.replace(".csv", "_processed.csv")
    df.to_csv(processed_csv_file, index=False)
    
    # fig, axs = plt.subplots(2, 2, figsize=(10,10))

    # axs[0, 0].plot(var, filtered_signal)
    # axs[0, 0].set_title(file_csv)
    # axs[0, 0].set_xlabel('Time (us)')
    # axs[0, 0].set_ylabel('Amplitude (V)')

    # axs[0, 1].plot(var, rectified_signal)
    # axs[0, 1].set_title(file_csv)
    # axs[0, 1].set_xlabel('Time (us)')
    # axs[0, 1].set_ylabel('Amplitude (V)')

    # axs[1, 0].plot(freq, fft_amp)
    # axs[1, 0].set_title(file_csv)
    # axs[1, 0].set_xlabel('Frequency (MHz)')
    # axs[1, 0].set_ylabel('Amplitude Power/Frequency (dB/Hz)')

    # axs[1, 1].plot(freq_psd, psd_one_side)
    # axs[1, 1].set_title(file_csv)
    # axs[1, 1].set_xlabel('Frequency (MHz)')
    # axs[1, 1].set_ylabel('Power/Frequency (dB/Hz)')

    # save_img = file_csv+".png"
    # plt.savefig(save_img)


# Move PSD to New Folder

In [6]:
os.chdir(load)
if not os.path.exists(load+"Process"):
        os.makedirs(load+"Process")
        print("'data' folder created")
        
for root, dirs, files in os.walk(load):
        for filename in files:
            if filename.endswith("_processed.csv"):
                original_file = os.path.join(root, filename)                
                if not "Process/" in original_file:
                    print("Moved '",original_file," to ","./Process/"+filename)
                    os.rename(original_file, "./Process/"+filename)

Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\A301_processed.csv  to  ./Process/A301_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\A302_processed.csv  to  ./Process/A302_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\A303_processed.csv  to  ./Process/A303_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\A304_processed.csv  to  ./Process/A304_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\A305_processed.csv  to  ./Process/A305_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\K301_processed.csv  to  ./Process/K301_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\K302_processed.csv  to  ./Process/K302_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\S301_processed.csv  to  ./Process/S301_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\S302_processed.csv  to  ./Process/S302_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\S303_processed.csv  to  ./Process/S303_processed.csv
Moved ' E:/Sekolah/Telkom/TA/LTSM/30%\S304_processed.csv  to  ./Process/S304_processed.csv

# Gabungkan Dataset menjadi satu

In [8]:
# Menggunakan glob untuk mendapatkan daftar file CSV
csv_files = glob.glob(proc_30)

# Membuat list kosong untuk menyimpan DataFrame dari setiap file CSV
dataframes = []

# Menggabungkan semua file CSV menjadi satu DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Menggabungkan DataFrame dalam list menjadi satu DataFrame utama
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('E:/Sekolah/Telkom/TA/LTSM/Process/Comb/1.csv', index=False)

In [9]:
# Menggunakan glob untuk mendapatkan daftar file CSV
csv_files = glob.glob(proc_50)

# Membuat list kosong untuk menyimpan DataFrame dari setiap file CSV
dataframes = []

# Menggabungkan semua file CSV menjadi satu DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Menggabungkan DataFrame dalam list menjadi satu DataFrame utama
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('E:/Sekolah/Telkom/TA/LTSM/Process/Comb/2.csv', index=False)

In [10]:
# Menggunakan glob untuk mendapatkan daftar file CSV
csv_files = glob.glob(proc_70)

# Membuat list kosong untuk menyimpan DataFrame dari setiap file CSV
dataframes = []

# Menggabungkan semua file CSV menjadi satu DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Menggabungkan DataFrame dalam list menjadi satu DataFrame utama
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('E:/Sekolah/Telkom/TA/LTSM/Process/Comb/3.csv', index=False)

In [11]:
# Menggunakan glob untuk mendapatkan daftar file CSV
csv_files = glob.glob(proc_90)

# Membuat list kosong untuk menyimpan DataFrame dari setiap file CSV
dataframes = []

# Menggabungkan semua file CSV menjadi satu DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Menggabungkan DataFrame dalam list menjadi satu DataFrame utama
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('E:/Sekolah/Telkom/TA/LTSM/Process/Comb/4.csv', index=False)

In [13]:
merge = 'E:/Sekolah/Telkom/TA/LTSM/Process/Comb/All.csv'

# Menggunakan glob untuk mendapatkan daftar file CSV
csv_files = glob.glob(combine)

# Membuat list kosong untuk menyimpan DataFrame dari setiap file CSV
dataframes = []

# Menggabungkan semua file CSV menjadi satu DataFrame
for i, file in enumerate(csv_files):
    df = pd.read_csv(file)
    df['label'] = f'label_{i+1}'
    dataframes.append(df)
# Menggabungkan DataFrame dalam list menjadi satu DataFrame utama
combined_df = pd.concat(dataframes, ignore_index=True)

# Menampilkan hasil gabungan
print(combined_df)

combined_df.to_csv(merge, index=False)

                 X         Y    label
0         5.000000 -0.035156  label_1
1         5.015625 -0.035156  label_1
2         5.031250 -0.039062  label_1
3         5.046875 -0.007812  label_1
4         5.062500  0.117188  label_1
...            ...       ...      ...
599035  199.921875 -0.003906  label_4
599036  199.937500 -0.003906  label_4
599037  199.953125  0.000000  label_4
599038  199.968750 -1.523438  label_4
599039  199.984375  1.519531  label_4

[599040 rows x 3 columns]


# Rename Label

In [14]:
data_df = pd.read_csv(merge)

label_mapping = {
    'label_1': 'Density 30%',
    'label_2': 'Density 50%',
    'label_3': 'Density 70%',
    'label_4': 'Density 90%'
}

data_df['label'] = data_df['label'].map(label_mapping)
data_df.to_csv(merge, index=False)

# Train Dataset

In [15]:
# Membaca dataset utama untuk pelatihan
data_df = pd.read_csv(merge)
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values

In [16]:
# Menggunakan LabelEncoder untuk mengubah label menjadi nilai numerik
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded_categorical = utils.to_categorical(y_encoded)
print(y_encoded_categorical.shape)

(599040, 4)


In [17]:
# Membagi dataset menjadi data pelatihan dan data validasi
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded_categorical, test_size=0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

## Metode LTSM

In [18]:
# Membangun model LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_encoded_categorical.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Melatih model dengan dataset utama
# model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

In [20]:
# model.save('All.h5')

## Load Model

In [21]:
model = load_model('All.h5')

# Predict Datates

In [22]:
# Membaca dataset yang ingin diprediksi
predict_df = pd.read_csv('E:/Sekolah/Telkom/TA/LTSM/Process/50%/K502_processed.csv')
X_predict = predict_df.iloc[:].values

In [23]:
# Memprediksi label menggunakan model yang telah dilatih
y_pred_encoded = model.predict(X_predict)
y_pred = np.argmax(y_pred_encoded, axis=1)
y_pred_label = label_encoder.inverse_transform(y_pred)

390/390 [==============================] - 2s 3ms/step


# Save Predict

In [24]:
result_df = predict_df.copy()
result_df['predicted_label'] = y_pred_label
result_df.to_csv('data_predict_predicted.csv', index=False)

# Result Predict

In [25]:
print(result_df['predicted_label'].value_counts())

predicted_label
Density 50%    8802
Density 90%    2710
Density 30%     591
Density 70%     377
Name: count, dtype: int64


In [26]:
# Membandingkan kemiripan label dengan dataset utama
similar_labels = []
for label in result_df['predicted_label'].unique():
    similarity = max(SequenceMatcher(None, label, unique_label).ratio() for unique_label in np.unique(y))
if similarity > 0.7:
    print("Data match :", label)

Data match : Density 50%


In [ ]:
Piye carane ben iso muncul 1 dataset tok